In [23]:
import zipfile
import os
import pandas as pd
import json


In [14]:
base_path = '/content/drive/MyDrive/курс 4.1/NLP/Project/data'
content_path = os.path.join(base_path, 'content')
archive_path = os.path.join(base_path, 'archive')

zip_paths = [os.path.join(archive_path, 'archive1.zip'),
             os.path.join(archive_path, 'archive2.zip'),
             os.path.join(archive_path, 'archive3.zip')]

for zip_path in zip_paths:
  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      zip_ref.extractall(content_path)

file1 = os.path.join(content_path, 'FA-KES-Dataset.csv')
file2 = os.path.join(content_path, 'politifact_factcheck_data.json')
file3 = os.path.join(content_path, 'Sarcasm_Headlines_Dataset_v2.json')

In [15]:
df = pd.read_csv(file1, encoding='ISO-8859-1')

df_FAKES_Dataset = pd.DataFrame()

df_FAKES_Dataset['title'] = df['article_title']
df_FAKES_Dataset['text'] = df['article_content']
df_FAKES_Dataset['source'] = df['source']
df_FAKES_Dataset['fake'] = df['labels'].apply(lambda x: 1 if x == 1 else 0)

print(df_FAKES_Dataset.head())


                                               title  \
0  Syria attack symptoms consistent with nerve ag...   
1  Homs governor says U.S. attack caused deaths b...   
2    Death toll from Aleppo bomb attack at least 112   
3        Aleppo bomb blast kills six Syrian state TV   
4  29 Syria Rebels Dead in Fighting for Key Alepp...   

                                                text source  fake  
0  Wed 05 Apr 2017 Syria attack symptoms consiste...    nna     0  
1  Fri 07 Apr 2017 at 0914 Homs governor says U.S...    nna     0  
2  Sun 16 Apr 2017 Death toll from Aleppo bomb at...    nna     0  
3  Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...    nna     0  
4  Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...    nna     0  


In [16]:
df = pd.read_json(file2, lines=True)
df_politifact_factcheck = pd.DataFrame()

df_politifact_factcheck['title'] = None
df_politifact_factcheck['text'] = df['statement']
df_politifact_factcheck['source'] = df['statement_source']
df_politifact_factcheck['fake'] = df['verdict'].apply(lambda x: 1 if x in ['true', 'mostly-true', 'half-true'] else 0)

print(df_politifact_factcheck.head())

  title                                               text      source  fake
0   NaN  John McCain opposed bankruptcy protections for...      speech     1
1   NaN  "Bennie Thompson actively cheer-led riots in t...  television     0
2   NaN  Says Maggie Hassan was "out of state on 30 day...        news     1
3   NaN  "BUSTED: CDC Inflated COVID Numbers, Accused o...        blog     0
4   NaN  "I'm the only (Republican) candidate that has ...  television     1


In [17]:
df = pd.read_json(file3, lines=True)
df_sarcasm = pd.DataFrame()

df_sarcasm['title'] = df['headline']
df_sarcasm['text'] = None
df_sarcasm['source'] = None
df_sarcasm['fake'] = df['is_sarcastic'].apply(lambda x: 'Sarcastic' if x == 1 else 'True')

print(df_sarcasm.head())

                                               title  text source       fake
0  thirtysomething scientists unveil doomsday clo...  None   None  Sarcastic
1  dem rep. totally nails why congress is falling...  None   None       True
2  eat your veggies: 9 deliciously different recipes  None   None       True
3  inclement weather prevents liar from getting t...  None   None  Sarcastic
4  mother comes pretty close to using word 'strea...  None   None  Sarcastic


In [18]:
df_combined = pd.concat([df_FAKES_Dataset, df_politifact_factcheck], ignore_index=True)
df_combined.drop_duplicates(inplace=True)
df_sarcasm.drop_duplicates(inplace=True)

save_path=os.path.join(base_path, 'arsenii_result.csv')
save_sarcastic_path=os.path.join(base_path, 'sarcastic.csv')

df_combined.to_csv(save_path, index=False, encoding='utf-8')
df_sarcasm.to_csv(save_sarcastic_path, index=False, encoding='utf-8')

In [19]:
# Порожні значення (NaN або порожні рядки)
empty_titles = df_combined[df_combined['title'].str.strip() == '']
empty_texts = df_combined[df_combined['text'].str.strip() == '']

# Неправильні типи значень (не рядки в колонках 'Title' та 'Text')
incorrect_titles = df_combined[~df_combined['title'].apply(type).eq(str)]
incorrect_texts = df_combined[~df_combined['text'].apply(type).eq(str)]

# Занадто короткі або довгі значення (для 'Title' і 'Text')
short_titles = df_combined[df_combined['title'].str.len() < 10]
short_texts = df_combined[df_combined['text'].str.len() < 10]

# Дублікати
duplicates = df_combined[df_combined.duplicated()]

# Виведення результатів
print(f"Кількість порожніх заголовків (title): {empty_titles.shape[0]}")
print("Порожні заголовки:")
print(empty_titles[['title']])

print(f"Кількість порожніх текстів (text): {empty_texts.shape[0]}")
print("Порожні тексти:")
print(empty_texts[['text']])

print(f"Кількість записів з неправильним типом в title: {incorrect_titles.shape[0]}")
print("Записи з неправильним типом в title:")
print(incorrect_titles[['title']])

print(f"Кількість записів з неправильним типом в text: {incorrect_texts.shape[0]}")
print("Записи з неправильним типом в text:")
print(incorrect_texts[['text']])

print(f"Кількість коротких заголовків (title): {short_titles.shape[0]}")
print("Короткі заголовки:")
print(short_titles[['title']])

print(f"Кількість коротких текстів (text): {short_texts.shape[0]}")
print("Короткі тексти:")
print(short_texts[['text']])

print(f"Кількість дублікатів: {duplicates.shape[0]}")
print("Дублікати:")
print(duplicates)


Кількість порожніх заголовків (title): 0
Порожні заголовки:
Empty DataFrame
Columns: [title]
Index: []
Кількість порожніх текстів (text): 0
Порожні тексти:
Empty DataFrame
Columns: [text]
Index: []
Кількість записів з неправильним типом в title: 21143
Записи з неправильним типом в title:
      title
804     NaN
805     NaN
806     NaN
807     NaN
808     NaN
...     ...
21951   NaN
21952   NaN
21953   NaN
21954   NaN
21955   NaN

[21143 rows x 1 columns]
Кількість записів з неправильним типом в text: 0
Записи з неправильним типом в text:
Empty DataFrame
Columns: [text]
Index: []
Кількість коротких заголовків (title): 0
Короткі заголовки:
Empty DataFrame
Columns: [title]
Index: []
Кількість коротких текстів (text): 0
Короткі тексти:
Empty DataFrame
Columns: [text]
Index: []
Кількість дублікатів: 0
Дублікати:
Empty DataFrame
Columns: [title, text, source, fake]
Index: []


In [20]:
#Два наших файли в 1
df1 = pd.read_csv(os.path.join(base_path, 'arsenii_result.csv'))
df2 = pd.read_csv(os.path.join(base_path, 'nika_result.csv'))
df = pd.concat([df1, df2], ignore_index=True)
save_path=os.path.join(base_path, 'general.csv')
df.to_csv(save_path, index=False, encoding='utf-8')